<a href="https://colab.research.google.com/github/TsamayaDesigns/codeDivision-automation-with-python/blob/main/automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive & Set Working Directory:**

In [1]:
# 1️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# 2️⃣ Set Working Directory
import os
working_dir = '/content/drive/MyDrive/codeDivision'
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

# 3️⃣ Confirm Current Working Directory
print(f"Current working directory is: {os.getcwd()}")

Mounted at /content/drive/
Current working directory is: /content/drive/MyDrive/codeDivision


**Query the Data:**

In [26]:
import re
import pandas as pd
from smtp import send
from datetime import datetime
from excel_tools import read_sheet
fields = ['ID', 'Start_time', 'Completion_time', 'User_Email', 'Name', 'Last_modified_time', 'full_name', 'Need_365', 'Mac_or_PC', 'Moodle_Email', 'Attending', 'Alternative', 'Council', 'Postcode', 'NeuroDivergent', 'Woman', 'Minority', 'Disabled', 'Refugee']
subs1 = read_sheet('EnrolmentForm1.xlsx', fields = fields)
subs2 = read_sheet('EnrolmentForm2.xlsx', fields = fields)


**Check Attendance:**

In [55]:
# Check if person is attending
def attendance(subs_list):
  attend_y = []
  attend_n = []

  for sub in subs_list:
      if str(sub.Attending).strip().lower() in ('yes', 'y'):
          attend_y.append(sub)
      else:
          attend_n.append(sub)

  print(f'Attendance Yes: {len(attend_y)} \nAttendance No: {len(attend_n)}')
  return attend_y, attend_n

attend_y1, attend_n1 = attendance(subs1)
attend_y2, attend_n2 = attendance(subs2)


Attendance Yes: 8 
Attendance No: 9
Attendance Yes: 4 
Attendance No: 5


**Email address validation:**

In [67]:
# Check if email address is valid
EMAIL_REGEX = re.compile(
    r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
)

def validate_emails(Moodle_Email):
  valid_emails = []
  invalid_emails = []

  if not EMAIL_REGEX.match(Moodle_Email.strip()):
    invalid_emails.append(f'Moodle_Email: {Moodle_Email.strip()}')
  else:
    valid_emails.append(f'Moodle_Email: {Moodle_Email.strip()}')

  return invalid_emails, valid_emails


# Print all valid emails
def get_valid_emails(subs_list, form_name):
  # Collect all valid emails
  form_valid_emails = []

  for sub in subs_list:
      _, valid_emails = validate_emails(sub.Moodle_Email)

      if valid_emails:
        form_valid_emails.append((sub.full_name, sub.Moodle_Email))

  print(f'Valid emails from {form_name}: {form_valid_emails}')
  return form_valid_emails

valid_emails_f1 = get_valid_emails(subs1, 'EnrolmentForm1.xlsx')
valid_emails_f2 = get_valid_emails(subs2, 'EnrolmentForm2.xlsx')


print()

# Print email address report (for each person)
def email_address_report(subs_list, form_name):
  print(f'Email address report {form_name}:')
  for sub in subs_list:
      invalid_emails, valid_emails = validate_emails(sub.Moodle_Email)

      if invalid_emails:
          print(sub.full_name, 'has an invalid email:', invalid_emails)

      if valid_emails:
          print(sub.full_name, 'has a valid email:', valid_emails)

email_address_report(subs1, '(EnrolmentForm1.xlsx)')
print()
email_address_report(subs2, '(EnrolmentForm2.xlsx)')

Valid emails from EnrolmentForm1.xlsx: [('Piet Pompies', 'piet@email.com'), ('Jan Pampoen', 'jan@email.com'), ('Koos Kakebeen', 'koos@email.com'), ('Peter Pumpkineater', 'peter@email.com'), ('Gert Coetzee', 'gert@google.com'), ('Jan Pierewiet', 'JanP@outlook.com'), ('Blake Turner', 'thompsonpeter@example.org'), ('Amy Tran', 'gpacheco@example.net'), ('Anna Marsh', 'turnerstephanie@example.com'), ('William Edwards', 'patrick79@example.com'), ('Thomas Dixon', 'usims@example.net'), ('Jonathan Morton', 'deborahcuevas@example.net'), ('Russell Frederick', 'markbutler@example.org'), ('John Bowman', 'samanthabrown@example.net'), ('Alyssa White', 'millerdouglas@example.org'), ('Katrina Cook', 'leemary@example.com')]
Valid emails from EnrolmentForm2.xlsx: [('Mike Hassett', 'mike@mail.com'), ('Ben Burger', 'ben@gmail.com'), ('Clive Greenwood', 'clive@trustytransport.co.za'), ('Coolhand Luke', 'coolhand@yahoo.com'), ('Steph Rossouw', 'steph@windhoek.co.na'), ('Gert Coetzee', 'gert@google.com'), ('J

**Office 365 Requirements:**

In [68]:
# Establish Office 365 requirements
o365_y = []
o365_n = []

for sub in subs:
    if sub.Need_365 == 'Yes' or sub.Need_365 == 'yes' or sub.Need_365 == 'Y' or sub.Need_365 == 'y':
        o365_y.append(sub)
    else:
        o365_n.append(sub)

print(f'Require 365 - Yes: {len(o365_y)} \nRequire 365 - No: {len(o365_n)}')

Require 365 - Yes: 7 
Require 365 - No: 10


**Available Dates:**

In [69]:
# Check alternative dates for people unable attend

for sub in subs:
  if sub.Attending.strip().lower() in ['no', 'n']:
    try:
      date_str = sub.Alternative

      if isinstance(date_str, str):
        date = datetime.strptime(date_str.strip(), '%d/%m/%Y')
      elif isinstance(date_str, datetime):
        date = date_str
      else:
        raise ValueError("The Alternative date is not a string or datetime object.")

      print(f'{sub.full_name}, will be able to attend on: {date.strftime("%Y/%m/%d")}.')

    except ValueError:
      print(f'{sub.full_name}, entered an invalid date: \"{sub.Alternative.strip()}\", and can be contacted on {sub.Moodle_Email.strip()} to confirm alternative date(s).')


Jan Pampoen, entered an invalid date: "NA", and can be contacted on jan@email.com to confirm alternative date(s).
Koos Kakebeen, entered an invalid date: "Help", and can be contacted on koos@email.com to confirm alternative date(s).
Pieter de Coning, will be able to attend on: 2025/10/05.
Amy Tran, will be able to attend on: 2025/07/10.
Thomas Dixon, will be able to attend on: 2025/07/17.
Russell Frederick, will be able to attend on: 2025/07/22.
John Bowman, will be able to attend on: 2025/08/23.
Alyssa White, will be able to attend on: 2025/07/10.
Katrina Cook, will be able to attend on: 2025/08/04.


**Duplicate Email Check:**

**Duplicate Email Check: (TEST)**

In [24]:
# # Check for and display duplicate emails
# duplicate_email = []
# non_duplicate_email = []

# subs_1 = subs
# subs_2 = subs2

# # Build a set of normalised emails from subs_1 & subs_2 for fast lookup
# subs1_emails = {sub.Moodle_Email.strip().lower() for sub in subs_1}
# subs2_emails = {sub.Moodle_Email.strip().lower() for sub in subs_2}

# # Iterate over subs_2 and check for duplicate emails
# for sub2 in subs2_emails:
#   if subs2_email in subs1_emails:
#       duplicate_email.append((sub2.full_name, sub2.Moodle_Email))
#   else:
#     non_duplicate_email.append((sub2.full_name, sub2.Moodle_Email))

# if len(subs2_emails) > len(subs1_emails):
#   if duplicate_email not in subs1_emails:
#     subs1_emails.append
#     # Check iteration and what happens to duplicates

# print(f'Duplicate emails in Form 2, that are already in Form 1: {duplicate_email}')
# print(f'New (non-duplicate) emails in Form 2: {non_duplicate_email}')


AttributeError: 'str' object has no attribute 'full_name'

**Export results to an Excel file:**

In [ ]:
# Create a DataFrame from the results
results_df = pd.DataFrame({
    'Duplicate Emails': pd.Series(duplicate_email),
    'Non-Duplicate Emails': pd.Series(non_duplicate_email)
})
print(results_df)
print()

# Write to Excel
results_df.to_excel('Email_Comparison_Results.xlsx', index=False)

print("Results exported to: 'Email_Comparison_Results.xlsx'")


**Sending Email:**

In [ ]:
# Prepare email body as a string
body_lines = []

body_lines.append('\nValid emails in Form 1:')
for name, email in all_valid_emails:
    body_lines.append(f'{name} - ({email})')

body_lines.append('\nNew Non-Duplicate emails in Form 2:')
for name, email in non_duplicate_email:
    body_lines.append(f'{name} - ({email})')

body_lines.append('\nDuplicate emails in Form 2, (already in Form 1):')
for name, email in duplicate_email:
    body_lines.append(f'{name} - ({email})')

email_body = '\n'.join(body_lines)

# Sending the Email
success = send(
    sender = "registrations@anysupport.co.uk",
    app_pass = "smss eolr jzjl pxvn",
    recip = "eugene@anysupport.co.uk",
    sub = "Test Email from Colab",
    body = email_body
)

print(email_body)